# Natural Language Processing

This notebook explains how you can collect data about the linguistic aspcects of the texts in your corpus using the Natural Languague Toolkit (nltk) and the `dtdpTdm` module. In a sense, the `dtdpTdm` module is a 'warpper' built around the `nltk` literature, to simplify the use of its modules and methods.  
 
If you have never used `nltk` before, you need to run the code below to install the relevant components.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

The `dtdp` has a method named `numberOfTokens()` which can count the total number of words in a given text. As a parameter, you need to mention the name of the file which contains the full text that you want to analyse. The method makes use of the `tokenise()` method which was explained in the notebook on the analysis of [Vocabulary](Vocabulary.ipynb). It functions as follows:

In [ ]:
import dtdpTdm as tdm
from os.path import join

dir = 'Corpus'
file = 'HeartofDarkness.txt'

nrTokens = tdm.numberOfTokens( join( dir , file ) )
print( '{} contains {} tokens.'.format( tdm.getTitle(file)  , nrTokens ) )


To test this code on one of your own texts, you obviously need to change the value of the `file` variable. 

The `sent_tokenize()` method from the `nltk` library can be used to divide a text into its separate sentences. If we divide the total number of words by the total number of sentences, we get a number which indicates the average length of the sentences, or the average number of words per sentence. In the `dtdpTdm` module, the method named `averageSentenceLength()` which carries out precisely this calculation. 

In [ ]:
import dtdpTdm as tdm
from os.path import join

dir = 'Corpus'
file = 'HeartofDarkness.txt'

title = tdm.getTitle(file) 
avgSentLength = tdm.averageSentenceLength( join( dir , file ) )

print( 'The sentences in "{}" contain {} words on average.'.format( title , avgSentLength ) )

Having created a list of all the tokens in a text, calculating the number of types is quit an easy step. It is simply a matter of deduplicating the list, as types are the distinct words that occur in a text. Data about the number
of types and the number of tokens can subsequently be used to calculate the type-token ratio, which gives an impression of the diversity of the vocabulary. Higher type-token ratios are indicative of a higher lexical diversity. Texts with relatively low type-token ratios are also more repetitive. 

The value of type-token ratio varies along with the total number of tokens of a text. In longer texts, it is more difficult for authors to keep on introducting new distinctive words. In all likelihood, the texts in your corpus will all have different lengths. To be able to compare the type-token ratios of these texts on an equal footing, we can choose to consider the type-token ratios of the first few words of the text only. We may decide, for instance, to limit our analysis to the first 1000 words of the texts, and to consider the number of unique words in these particular segments only. 

Type-token ratios can be calculated using the `typeTokenRatio()` method, implemented in the `dtdpTdm` module. This method requires two parameters: a filename and a number. The number which needs to be supplied as a second parameter specifies the number of words to be taken into consideration. The value that you need to work with obviously depends on the nature of your corpus. As a value for this 'cap', you may choose to work with the number of tokens of the shortest text in your corpus. 

In [ ]:
import os
import re

for file in os.listdir(dir):
    if re.search( 'txt$' , file ):
        tokens = tdm.typeTokenRatio( join( dir , file) , 1000 )
        print( '{}: {}'.format( tdm.getTitle(file) , tokens ) )

Data about the syntactic categories of words can be produced by making use of part of speech
(POS) taggers. Python's `nltk` library includes a method called `pos_tag()`, which takes a list of words as input, and which can create a list of words and tags as output. The tags that are produced are based on the list created in the [Penn Treebank Project](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). 

The `showPosTags()` method of the `dtdpTdm` module makes use of the POS tagger which is distributed as part of `nltk`. Next to a filename, it expects a Python list of tags. Adjectives, for instance, can be tagged as 'JJ' (regular adjecives), 'JJR' (adjectives in the comparative form) or 'JJS' (superatives). This method ruturns a new list of all the adjectives which were labelled with either one of the tags in the tag list. 

In [ ]:
import dtdpTdm

dir = 'Corpus'
file = 'HeartofDarkness.txt'

tags = [ 'JJ' , 'JJR', 'JJS' ]
adjectives = tdm.showPosTags( join( dir , file ) , tags )

for a in adjectives:
    print(a)


The `countPosTags()` method is very similar. Instead of returning the actual list, however, it only returns the number of tokens in the list.  

Pythons's `nltk` library does not include a method that can count the number of syllables in a word, surprisingly. To address this lacuna, a new method has been programmed which attempts to perform this task, on the basis of regular expressions. This method is called `countSyllables()`. As its own only parameter, the method demands a single English word. A demonstration is given below.

In [ ]:
print( tdm.countSyllables("beauty") )
print( tdm.countSyllables("believe") )
print( tdm.countSyllables("university") )

When we divide the total number of syllables in a text by the total number of tokens, the number that results gives an indication of the average length of the words (if measured in the number of syllables).

In [ ]:
title = tdm.getTitle(file)
syll = tdm.averageNumberOfSyllables( join( dir , file ) ) 

print ( 'The words in {} contain {} syllables on average.'.format( title , syll ))

Data on the average sentence lengths and on the average word lengths are commonly used in formulae developed to assess the overall complexity of a text. The Flesch–Kincaid Grade Level Formula, for instance, uses these two numbers in a complicated formula which ultimately expresses the complexity of a texts as the number of years of education that is required to comprehend the text. For more information, see, for instance: https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests. 

In [ ]:
fk = tdm.fleschKincaid( join( dir , file ) )
title = tdm.getTitle( file )

print( 'To fully understand {}, you need to have followed {} years of education. '.format( title,fk ) )

The code below brings together all the methods that have been discussed to this point. It creates a CSV file named 'ntlk.csv'. For all the texts in your corpus, it creates data about the total number of tokens, the average sentence length, the type-token ratio, the adjective-token ratio (i.e. the number of adjectives divided by the total number of words), the adverb-token ratio, the average number of syllables per word and result of the Flesch–Kincaid Grade Level Formula. 

If your texts are long or if there are many texts in your corpus, running the code may also take quite a while. The print statements have been added to give you updates on the progress during the data creation process.

In [ ]:
import os
from os.path import join
import re
import dtdpTdm as tdm

dir = 'Corpus'
texts = []
pos = []

out = open( 'nltk.csv' , 'w' ,  encoding= 'utf-8' )

out.write( 'title,tokens,avgSentLength,ttr,adjectives,adverbs,persPronouns,nrSyllables,fk\n' )


for file in os.listdir(dir):
    if re.search( 'txt$' , file ):
        print( '\n\nCollecting data for ' + file + ' ... ')
        out.write( tdm.getTitle( file ) + ',' )
        
        path = join( dir , file )
        print("Counting number of tokens ...  ")
        tokens = tdm.numberOfTokens( path )
        out.write( '{},'.format( tokens ) )
        print("Calculating average sentence length ...  ")
        out.write( '{},'.format( tdm.averageSentenceLength( path )  ) )
        print("Counting of type-token ratio ...  ")
        out.write( '{},'.format( tdm.typeTokenRatio( path , 1000 ) ) )
        print("Counting number of adjectives ...  ")
        adjectives = [ 'JJ' , 'JJR' , 'JJS' ]
        out.write( '{},'.format( tdm.countPosTags( path , adjectives ) / tokens ) )
        print("Counting number of adverbs ...  ")
        adverbs = [ 'RB' , 'RBR' , 'RBS' ]
        out.write( '{},'.format( tdm.countPosTags( path , adjectives ) / tokens ) )
        print("Counting number of personal pronouns ...  ")
        personalPronouns = [ 'PRP' ]
        out.write( '{},'.format( tdm.countPosTags( path , personalPronouns ) / tokens ) )
        out.write( '{},'.format( tdm.averageNumberOfSyllables( path ) ) )  
        print("Calculating Flesch–Kincaid Grade Level Formula ...  ")
        out.write( '{}'.format( tdm.fleschKincaid( path ) ) )      

        out.write( '\n' )
    
out.close()    

print("\n\nDone!")


Using the CSV that was just created, a number of basic visualisations can be produced. Such visualisation should enable you to see the differences between the texts in your corpus, compared on the metrics you have collected using this notebook. Next to studying the differences between individual texts, it may also be interesting to study the broader differences between the various categories in your corpus. You may be working, for instance, with texts from different genres, periods, authors or text types. In the notebook named [Introduction](Introduction.ipynb), you learned how you can create a metadata file in the CSV format, offering information about these various categories. 

If you merge this metadata file with the nltk CSV file that you have just created, using the following code, the data about these categories can also be used in the visualisations. If you have changed the name of the columns `class` in the metadata CSV, you clearly need to change the column name in the code below as well. 

In [ ]:
import pandas as pd

df = pd.read_csv( 'nltk.csv' )
metadata = pd.read_csv( 'metadata.csv' )
df['class'] = metadata['class']

df.to_csv (r'dataset.csv', index = False, header=True) 

Having data about the different categories, we can create visualisations in which all the texts that share a certain category are also shown in a specific colour. If this is what we want to do, it can be useful to create a dictionary beforehand in which the different classes are all associated with a specific colour. The code in following cell can do this. It firstly identifies all the unique values in the column named `class`. Next, it assigns colours to these classes, by picking specific colour codes from the list named `colours`, which is also defined below. By default, the list only contains five items. If you have more categories, you need to add colours to the list. 

You are free to change the colours, of course, using, for instanc, a [colour picker](https://htmlcolors.com/google-color-picker). 




In [ ]:
df = pd.read_csv( 'dataset.csv' )

colours = [ '#09349E' , '#D1AC32' , '#C70C2B' , '#6AD964' , '#A640E6' ]

classColours = dict()

unique_categories = list( set( df['class'] ) )
if len( unique_categories ) <= len(colours):
    for u in range( len( unique_categories ) ):
        classColours[ unique_categories[u] ] = colours[u]
else:
    print("You have more than five categories. You need to add colours to the list!")
    


The code in the following cell creates a simple bar chart. The variable named `x` specifies the values that need to be shown on the X-axis. The variable named `y` does the same for the Y-axis. 

If you mention the column `'title'` as value of `x`, the X-axis will obviously list the titles of all the texts in you corpus. If you have created a `classColours` dictionary, the colours of the bars will offer information about the categories assigned to the various texts.


For the Y-axis, you can select all the values of the other columns: 

* tokens
* avgSentLength
* ttr
* adjectives
* adverbs
* nrSyllables
* fk



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd

df = pd.read_csv( 'dataset.csv' , index_col=False , decimal = '.' )

x = 'title'
y = 'avgSentLength'

colours = []
for category in df['class']:
    colours.append( classColours[category] )

fig = plt.figure( figsize=( 10 , 10 ) )
ax = plt.axes()


bar_width = 0.45
opacity = 0.8

ax.bar( df[x] , df[y] , width = bar_width, alpha = opacity , color = colours  )

plt.xticks(rotation= 90)


patchList = []
for key in classColours:
    data_key = mpatches.Patch(color=classColours[key], label=key)
    patchList.append(data_key)
    
plt.legend(handles=patchList , shadow=True, fontsize='large' , frameon = True )


ax.set_xlabel( x  , fontsize= 12)
ax.set_ylabel( y , fontsize = 12 )
ax.set_title( 'Title for bar chart' , fontsize=20 )

#plt.ylim( 0 , 0.175 )   


plt.show()
# plt.savefig( 'ntlk-barchart.TIFF')


The cell below creates a scatter plot, which can be used to compare the values of two variables. The two variables to be plotted are defined by the two variables named 'x' and 'y'. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as mpatches

df = pd.read_csv( 'dataset.csv' , index_col=False , decimal = '.' )

colours = []
for category in df['class']:
    colours.append( classColours[category] )

        
plt.style.use('seaborn-whitegrid')

x = 'adverbs'
y = 'ttr'


fig = plt.figure( figsize = ( 8,8 ))
ax = plt.axes()


ax.scatter(  df[x]  , df[y] , alpha=0.8,  s=80 , c = colours )



for index, row in df.iterrows():
    plt.text( row[x], row[y] , row['title'] , fontsize=12)
    

patchList = []
for key in classColours:
    data_key = mpatches.Patch(color=classColours[key], label=key)
    patchList.append(data_key)
    

plt.legend(handles=patchList , shadow=True, fontsize='large' , frameon = True )
    
ax.set_xlabel( x , fontsize = 16 )
ax.set_ylabel( y  , fontsize = 16 )

plt.show()

The graphs that can be created using the Python code that was explained in this notebook can hopefully help you to answer some of the questions that you have formulated in your research project. The approaches that are discussed in the notebook named [Lexicons](Lexicons.ipynb) may help you to create even more useful data. 